# Binance Client Test Notebook

In [ ]:
import sys
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import threading
import time

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from clients.binance_client import BinanceClient

## Initialize Binance Client

In [ ]:
# Initialize client
client = BinanceClient(api_key=os.getenv('binance_api_key'), api_secret=os.getenv('binance_api_secret'), leverage=1, verbose=True)


## Fetch Current Funding Rates

In [ ]:
symbols = ['BTC/USDT:USDT', 
           'ETH/USDT:USDT', 
           'SOL/USDT:USDT', 
           'XRP/USDT:USDT', 
           'BNB/USDT:USDT', 
           'DOGE/USDT:USDT', 
           '1000PEPE/USDT:USDT', 
           'TON/USDT:USDT', 
           'AVAX/USDT:USDT', 
           'WIF/USDT:USDT', 
           'MATIC/USDT:USDT', 
           'AAVE/USDT:USDT', 
           'LINK/USDT:USDT', 
           'SUI/USDT:USDT', 
           'ADA/USDT:USDT']
current_rates = client.get_current_funding_rates(symbols)
print("Current Funding Rates:")
display(current_rates)

## Visualize Current Funding Rates

In [ ]:
plt.figure(figsize=(10, 6))
current_rates_sorted = current_rates.sort_values(by='Annualized Funding Rate')
sns.barplot(x='Symbol', y='Annualized Funding Rate', data=current_rates_sorted)
plt.title('Current Annualized Funding Rates')
plt.xticks(rotation=45)
plt.show()

## Fetch current positions and prices on Future and Spot

In [ ]:
# Symbol for basis trade
symbol = 'LINK/USDT'

# Fetch spot position (last 24 hours by default)
spot_position = client.fetch_spot_position(symbol)
print("Spot Position (last 24 hours):")
display(pd.DataFrame([spot_position]))

# If you want to change the time window, you can do so like this:
# spot_position_48h = client.fetch_spot_position(symbol, time_window=48)
# print("Spot Position (last 48 hours):")
# display(pd.DataFrame([spot_position_48h]))

In [ ]:
# Fetch futures position
futures_position = client.fetch_futures_position(symbol)
print("Futures Position:")
display(pd.DataFrame([futures_position]))

In [ ]:
# Calculate basis (sell fut, buy spot)
if spot_position and futures_position:
    basis = futures_position['current_price'] - spot_position['current_price']
    basis_percentage = (basis / spot_position['current_price']) * 100
    print(f"Current Basis: {basis:.2f} USDT ({basis_percentage:.2f}%)")


In [ ]:
# Fetch current funding rate
funding_rates = client.get_current_funding_rates([symbol])
print("Current Funding Rate:")
display(funding_rates)

In [ ]:
# Visualize the basis trade
if spot_position and futures_position:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
    
    # Plot positions
    ax1.bar(['Spot', 'Futures'], [spot_position['amount'], futures_position['amount']], color=['blue', 'orange'])
    ax1.set_title('Spot vs Futures Position Size')
    ax1.set_ylabel('Amount')
    
    # Plot prices
    ax2.bar(['Spot Entry', 'Spot Current', 'Futures Entry', 'Futures Current'], 
            [spot_position['entry_price'], spot_position['current_price'], 
             futures_position['entry_price'], futures_position['current_price']],
            color=['lightblue', 'blue', 'orange', 'red'])
    ax2.set_title('Entry vs Current Prices')
    ax2.set_ylabel('Price (USDT)')
    
    # Plot last trade details
    ax3.bar(['Last Trade Amount'], [spot_position['last_trade_amount']], color='green')
    ax3.set_title(f"Last Trade Details ({spot_position['last_trade_side']} at {spot_position['last_trade_time']})")
    ax3.set_ylabel('Amount')
    
    plt.tight_layout()
    plt.show()

## Backtest basis trade

In [ ]:
symbol = 'REEF/USDT'

# Get the last 1000 minutes of basis spread data
basis_spread_df = client.calculate_basis_spread(symbol)
basis_spread_df.to_csv('basis_spread.csv', index=False)

# Print the last 10 rows
print(basis_spread_df.tail(10))

# Calculate average basis spread
#average_spread = basis_spread_df['basis_spread_percentage'].mean()
#print(f"Average basis spread over the last 1000 minutes: {average_spread:.4f}%")

In [ ]:
symbols_to_monitor = ['REEFUSDT', '1000SATSUSDT', 'SUIUSDT', 'BAKEUSDT', 'ADAUSDT']

# Start WebSocket streams
client.start_websocket_streams(symbols_to_monitor)

# Start monitoring basis spreads with custom settings
threading.Thread(target=client.monitor_basis_spreads, args=(symbols_to_monitor, 0.004, -0.0005, 5, 300, False)).start()

# Keep the main thread running
while True:
    time.sleep(1)

## Live trade basis trade

In [ ]:
symbols_to_monitor = ['ARUSDT', '1000SATSUSDT', 'BAKEUSDT', 
                      'ONEUSDT', 'ALPACAUSDT', 'TLMUSDT', 
                      'PENDLEUSDT', 'SUNUSDT', 'ACEUSDT', 
                      'VIDTUSDT', 'MBOXUSDT', 'SYSUSDT',
                      'PEOPLEUSDT', 'WIFUSDT', 'RDNTUSDT']
#symbols_to_monitor = ['ALPACAUSDT']

# Start WebSocket streams
client.start_websocket_streams(symbols_to_monitor)

# Start monitoring basis spreads
threading.Thread(target=client.monitor_basis_spreads, args=(
    symbols_to_monitor,
    0.007,               # entry_threshold
    0.0,                 # exit_threshold
    5,                   # interval
    60,                  # update_interval
    False,               # log_all_prices
    True,                # execute_trades
    15,                   # trade_amount_usd
    0.002                # slippage
)).start()

# Keep the main thread running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping the monitor...")
finally:
    # Stop WebSocket streams
    client.stop_websocket_streams()

2024-09-06 18:44:16,083 - INFO - 
Current spreads at 2024-09-06 18:44:16:
2024-09-06 18:44:16,085 - INFO - ARUSDT: -0.1843%
2024-09-06 18:44:16,086 - INFO - 1000SATSUSDT: -0.0322%
2024-09-06 18:44:16,086 - INFO - BAKEUSDT: -0.0763%
2024-09-06 18:44:16,088 - INFO - ONEUSDT: -0.0943%
2024-09-06 18:44:16,094 - INFO - ALPACAUSDT: -0.1054%
2024-09-06 18:44:16,098 - INFO - TLMUSDT: -0.1828%
2024-09-06 18:44:16,098 - INFO - PENDLEUSDT: -0.0682%
2024-09-06 18:44:16,099 - INFO - SUNUSDT: -0.0980%
2024-09-06 18:44:16,100 - INFO - ACEUSDT: -0.1009%
2024-09-06 18:44:16,100 - INFO - VIDTUSDT: -0.0203%
2024-09-06 18:44:16,101 - INFO - MBOXUSDT: 0.0000%
2024-09-06 18:44:16,101 - INFO - SYSUSDT: 0.0000%
2024-09-06 18:44:16,102 - INFO - PEOPLEUSDT: 0.0068%
2024-09-06 18:44:16,103 - INFO - WIFUSDT: -0.0881%
2024-09-06 18:44:16,103 - INFO - RDNTUSDT: -0.2178%


2024-09-06 18:45:16,144 - INFO - 
Current spreads at 2024-09-06 18:45:16:
2024-09-06 18:45:16,146 - INFO - ARUSDT: -0.1843%
2024-09-06 18:45:16,146 - INFO - 1000SATSUSDT: -0.0322%
2024-09-06 18:45:16,147 - INFO - BAKEUSDT: -0.0763%
2024-09-06 18:45:16,147 - INFO - ONEUSDT: -0.0943%
2024-09-06 18:45:16,148 - INFO - ALPACAUSDT: -0.1054%
2024-09-06 18:45:16,152 - INFO - TLMUSDT: -0.1828%
2024-09-06 18:45:16,154 - INFO - PENDLEUSDT: -0.0682%
2024-09-06 18:45:16,154 - INFO - SUNUSDT: -0.0980%
2024-09-06 18:45:16,155 - INFO - ACEUSDT: -0.1009%
2024-09-06 18:45:16,156 - INFO - VIDTUSDT: -0.0203%
2024-09-06 18:45:16,156 - INFO - MBOXUSDT: 0.0000%
2024-09-06 18:45:16,158 - INFO - SYSUSDT: 0.0000%
2024-09-06 18:45:16,160 - INFO - PEOPLEUSDT: 0.0068%
2024-09-06 18:45:16,160 - INFO - WIFUSDT: -0.0881%
2024-09-06 18:45:16,161 - INFO - RDNTUSDT: -0.2178%


2024-09-06 18:46:16,211 - INFO - 
Current spreads at 2024-09-06 18:46:16:
2024-09-06 18:46:16,212 - INFO - ARUSDT: -0.1843%
2024-09-06 18:46:16,213 - INFO - 1000SATSUSDT: -0.0322%
2024-09-06 18:46:16,213 - INFO - BAKEUSDT: -0.0763%
2024-09-06 18:46:16,214 - INFO - ONEUSDT: -0.0943%
2024-09-06 18:46:16,214 - INFO - ALPACAUSDT: -0.1054%
2024-09-06 18:46:16,214 - INFO - TLMUSDT: -0.1828%
2024-09-06 18:46:16,216 - INFO - PENDLEUSDT: -0.0682%
2024-09-06 18:46:16,216 - INFO - SUNUSDT: -0.0980%
2024-09-06 18:46:16,217 - INFO - ACEUSDT: -0.1009%
2024-09-06 18:46:16,217 - INFO - VIDTUSDT: -0.0203%
2024-09-06 18:46:16,218 - INFO - MBOXUSDT: 0.0000%
2024-09-06 18:46:16,218 - INFO - SYSUSDT: 0.0000%
2024-09-06 18:46:16,219 - INFO - PEOPLEUSDT: 0.0068%
2024-09-06 18:46:16,220 - INFO - WIFUSDT: -0.0881%
2024-09-06 18:46:16,220 - INFO - RDNTUSDT: -0.2178%


2024-09-06 18:47:16,276 - INFO - 
Current spreads at 2024-09-06 18:47:16:
2024-09-06 18:47:16,277 - INFO - ARUSDT: -0.1843%
2024-09-06 18:47:16,278 - INFO - 1000SATSUSDT: -0.0322%
2024-09-06 18:47:16,279 - INFO - BAKEUSDT: -0.0763%
2024-09-06 18:47:16,280 - INFO - ONEUSDT: -0.0943%
2024-09-06 18:47:16,280 - INFO - ALPACAUSDT: -0.1054%
2024-09-06 18:47:16,281 - INFO - TLMUSDT: -0.1828%
2024-09-06 18:47:16,282 - INFO - PENDLEUSDT: -0.0682%
2024-09-06 18:47:16,282 - INFO - SUNUSDT: -0.0980%
2024-09-06 18:47:16,283 - INFO - ACEUSDT: -0.1009%
2024-09-06 18:47:16,284 - INFO - VIDTUSDT: -0.0203%
2024-09-06 18:47:16,285 - INFO - MBOXUSDT: 0.0000%
2024-09-06 18:47:16,286 - INFO - SYSUSDT: 0.0000%
2024-09-06 18:47:16,289 - INFO - PEOPLEUSDT: 0.0068%
2024-09-06 18:47:16,289 - INFO - WIFUSDT: -0.0881%
2024-09-06 18:47:16,291 - INFO - RDNTUSDT: -0.2178%


2024-09-06 18:48:16,339 - INFO - 
Current spreads at 2024-09-06 18:48:16:
2024-09-06 18:48:16,341 - INFO - ARUSDT: -0.1843%
2024-09-06 18:48:16,343 - INFO - 1000SATSUSDT: -0.0322%
2024-09-06 18:48:16,344 - INFO - BAKEUSDT: -0.0763%
2024-09-06 18:48:16,349 - INFO - ONEUSDT: -0.0943%
2024-09-06 18:48:16,352 - INFO - ALPACAUSDT: -0.1054%
2024-09-06 18:48:16,355 - INFO - TLMUSDT: -0.1828%
2024-09-06 18:48:16,356 - INFO - PENDLEUSDT: -0.0682%
2024-09-06 18:48:16,357 - INFO - SUNUSDT: -0.0980%
2024-09-06 18:48:16,358 - INFO - ACEUSDT: -0.1009%
2024-09-06 18:48:16,358 - INFO - VIDTUSDT: -0.0203%
2024-09-06 18:48:16,359 - INFO - MBOXUSDT: 0.0000%
2024-09-06 18:48:16,360 - INFO - SYSUSDT: 0.0000%
2024-09-06 18:48:16,361 - INFO - PEOPLEUSDT: 0.0068%
2024-09-06 18:48:16,362 - INFO - WIFUSDT: -0.0881%
2024-09-06 18:48:16,362 - INFO - RDNTUSDT: -0.2178%


## Triangular arbitrage

In [ ]:
BTCUSDT = 59493.45
ETHBTC = 0.04245
ETHUSDT = 2518.67

USDT = 15
USDT_1 = 15 / BTCUSDT
#print(USDT_1)

USDT_2 = USDT_1 / ETHBTC
#print(USDT_2)

USDT_3 = USDT_2 * ETHUSDT
print(USDT_3)



In [ ]:
#e.g. symbols 'ARUSDT', '1000SATSUSDT', 'BAKEUSDT', 'ONEUSDT', 'ALPACAUSDT', 'TLMUSDT', 
# 'PENDLEUSDT', 'SUNUSDT', 'ACEUSDT', 'VIDTUSDT', 'MBOXUSDT', 'SYSUSDT','PEOPLEUSDT', 'WIFUSDT', 'RDNTUSDT'

client.monitor_triangular_arbitrage(
    # Need to be in the right order!!
    symbol_triplet=['BTCUSDT', 'ETHBTC', 'ETHUSDT'],
    trades=['buy', 'buy', 'sell'],
    profit_threshold=0.002,  # 0.2% profit threshold
    interval=5,
    update_interval=10,
    execute_trades=True,
    trade_amount=15.0,
    max_slippage=0.001,
)

## Get OHLCV data

In [ ]:
symbol = 'BTC/USDT:USDT'
start_time = datetime.now() - timedelta(days=30)  # Last 30 days
ohlcv_data = client.get_ohlcv(symbol, timeframe='1d', since=start_time, limit=30)
display(ohlcv_data)

## Fetch Historical Funding Rates

In [ ]:
start_time = datetime.now() - timedelta(days=30)
start_time.timestamp() * 1000

In [ ]:
# Fetch Historical Funding Rates
symbol = 'BTC/USDT:USDT'
# Define start_time as a datetime object
start_time = datetime.now() - timedelta(days=30)
# Define end_time as the current datetime
end_time = datetime.now()

# Pass both datetime objects to the method
historical_rates = client.get_historical_funding_rates(symbol, start_time, end_time)
print("Historical Funding Rates:")
display(historical_rates.head())

## Visualize Historical Funding Rates

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(historical_rates['datetime'], historical_rates['annualized_rate'])
plt.title(f'Historical Annualized Funding Rates for {symbol}')
plt.xlabel('Date')
plt.ylabel('Annualized Funding Rate (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## Fetch OHLCV Data

In [ ]:
ohlcv_data = client.get_ohlcv(symbol, '1d', since=start_time)
print("OHLCV Data:")
display(ohlcv_data.head())

## Visualize OHLCV Data

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ohlcv_data['datetime'], ohlcv_data['close'])
plt.title(f'Closing Prices for {symbol}')
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## Analyze Correlation Between Price and Funding Rate

In [ ]:
# Merge OHLCV and funding rate data
merged_data = pd.merge(ohlcv_data, historical_rates, left_on='datetime', right_on='datetime', how='inner')

# Calculate correlation
correlation = merged_data['close'].corr(merged_data['annualized_rate'])
print(f"Correlation between price and funding rate: {correlation:.2f}")

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(merged_data['close'], merged_data['annualized_rate'])
plt.title(f'Price vs Funding Rate for {symbol}')
plt.xlabel('Price (USDT)')
plt.ylabel('Annualized Funding Rate (%)')
plt.grid(True)
plt.show()